<img src="QuSCo_Logo_CMYK.jpg" alt="Here should be the qusco logo!" width="500">

---

In [ ]:
import numpy as np
import scipy
import matplotlib
import matplotlib.pylab as plt
import krotov
import qutip
from exercise_03_utils import *

---

# Exercises

[Model](#Model)

[Exercise 3.1: Implementing the System ](#Exercise-3.1:-Implementing-the-System)  
[Exercise 3.2: Objective](#Exercise-3.2:-Objective)  
[Exercise 3.3: Shaping our guess pulses](#Exercise-3.3:-Shaping-our-guess-pulses)  
[Exercise 3.4: Specifying the pulse options](#Exercise-3.4:-Specifying-the-pulse-options)  
[Exercise 3.5: The optimization](#Exercise-3.5:-The-optimization)  
[Exercise 3.6: Analysing the results](#Exercise-3.6:-Analysing-the-results)

[Bonus Exercise: Adding dissipation](#Bonus-exercise)

 
---


# Model


$\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2} \mid
#2\vphantom{#1}\right\rangle}
\newcommand{Ketbra}[2]{\left\vert#1\vphantom{#2}
\right\rangle \hspace{-0.2em} \left\langle #2\vphantom{#1}\right\vert}
\newcommand{e}[1]{\mathrm{e}^{#1}}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}
\newcommand{toP}[0]{\omega_{12}}
\newcommand{toS}[0]{\omega_{23}}
\newcommand{oft}[0]{\left(t\right)}$
Our model consists of a "Lambda system" as shown below.
These levels interact with two pulses with the base frequency $\omega_{\mathrm{P}}$ ("Pump"-pulse) and $\omega_{\mathrm{S}}$ ("Stokes"-pulse), respectively. These pulses have time-dependent envelopes 
\begin{align*}
\epsilon_{\mathrm{P}}(t) &= \frac{\Omega_{\mathrm{P}}^{(1)}(t)}{\mu_{12}}\cos({\omega_{\mathrm{P}}}t)
                           +\frac{\Omega_{\mathrm{P}}^{(2)}(t)}{\mu_{12}}\sin({\omega_{\mathrm{P}}}t) \\
\epsilon_{\mathrm{S}}(t) &= \frac{\Omega_{\mathrm{S}}^{(1)}(t)}{\mu_{23}}\cos({\omega_{\mathrm{S}}}t)
                           +\frac{\Omega_{\mathrm{S}}^{(2)}(t)}{\mu_{23}}\sin({\omega_{\mathrm{S}}}t),
\end{align*}
With the coupling strength $\mu_{ij}$ between the levels $i$ and $j$.
The frequencies are chosen, such that they are close to the transition frequencies $\ket{1}\rightarrow\ket{2}$ ($\omega_{12}$) and $\ket{3} \rightarrow\ket{2}$ ($\omega_{32}$).
To represent the evolution in the rotating frame, we use the free evolution operator

\begin{equation*}
U_{0} =  \begin{pmatrix}
\e{-i(\omega_2-\omega_{\mathrm{P}})t} & 0 & 0 \\
0 & \e{-i \omega_2 t} & 0 \\
0 & 0 & \e{-i(\omega_2-\omega_{\mathrm{S}})t} 
\end{pmatrix},
\end{equation*}

with $\omega_2 = E_2/\hbar$ the frequency fo the energy level $\ket{2}$.
With this we can transform the Hamiltonian of the system 
\begin{equation*}
\hat{H} = \hat{H}_{0} + \hat{H}_{1}  =  \begin{pmatrix}
E_1 & 0 & 0 \\
0 & E_2 & 0 \\
0 & 0 & E_3
\end{pmatrix}
-
\begin{pmatrix}
0 & \mu_{12}\epsilon_{\mathrm{P}}(t) & 0 \\
\mu_{12}\epsilon_{\mathrm{P}}(t) & 0 & \mu_{23}\epsilon_{\mathrm{P}}(t) \\
0 & \mu_{23}\epsilon_{\mathrm{P}}(t) & 0
\end{pmatrix},
\end{equation*}
to 
\begin{equation*}
\hat{H}' =  \hbar \begin{pmatrix}
-\Delta_{\mathrm{P}} & \Omega^{\ast}_{\mathrm{P}}(t) & 0 \\
\Omega_{\mathrm{P}}(t) & 0 & \Omega^{\ast}_{\mathrm{S}}(t) \\
0 & \Omega_{\mathrm{S}}(t) & -\Delta_{\mathrm{S}}
\end{pmatrix},
\end{equation*}
with $\Delta_{\mathrm{P}} = E_1 + \omega_{\mathrm{P}} - E_2$ and $\Delta_{\mathrm{S}} = E_3 + \omega_{\mathrm{S}} - E_2$.
The envelopes become complex with $\Omega_{\mathrm{P}} = \Omega^{(1)}_{\mathrm{P}} + i\Omega^{(2)}_{\mathrm{P}}$ and $\Omega_{\mathrm{S}} = \Omega^{(1)}_{\mathrm{S}} + i\Omega^{(2)}_{\mathrm{S}}$.

In the following, we will optimize the real and imaginary part of $\Omega_{\mathrm{S}}$ and $\Omega_{\mathrm{P}}$ independently. 





<img src="tikzpics/energylevels.png" alt="Lambda system considered in this notebook" width="500">

---

# Exercise 3.1: Implementing the System

&nbsp;&nbsp;&nbsp;
**a)** Set up H0 as described above.

In [ ]:
#Parameters
E1 = 0.
E2 = 10.
E3 = 5.
ω_P = 9.5
ω_S = 4.5
Ω_init = 5.
tlist = np.linspace(0.,5,500)

In [ ]:
H0 =  '---'

&nbsp;&nbsp;&nbsp;
**b)** Set up the real and imaginary part of $\mathrm{H}_{1P}$ and $\mathrm{H}_{1S}$, according to the definition above.

In [ ]:
# Qutip objects holding the real and imaginary part of the Hamiltoninan H_1P
H1P_re = '---'  
H1P_im = '---'
# initial funtions, which will later be contorls
ΩP_re = lambda t, args: Ω_init
ΩP_im = lambda t, args: Ω_init

In [ ]:
# Qutip objects holding the real and imaginary part of the Hamiltoninan H_1S
H1S_re = '---'
H1S_im = '---'
# initial funtions, which will later be contorls
ΩS_re = lambda t, args: Ω_init    
ΩS_im = lambda t, args: Ω_init

&nbsp;&nbsp;&nbsp;
**c)** Specify the initial $\big(\Psi_0 = \ket{1}\big)$ and the target state $\big(\Psi_1 = \ket{3}\big)$ for the optimization.

In [ ]:
"""Initial and target states"""
psi0 = '---'
psi1 = '---'    

In [ ]:
#q1c.hint()

&nbsp;&nbsp;&nbsp;
**d)** Define the overall Hamiltonian by combining the partial Hamiltoians defined in *b)* with the corresponding control.

In [ ]:
"""Final Hamiltonian"""
Ham = [H0, "..."]

In [ ]:
#q1d.hint()

&nbsp;&nbsp;&nbsp;
**e)** Finally specify the projectors $\hat{P}_i = \ket{i}\bra{i}$.

In [ ]:
proj1 = '---'
proj2 = '---'
proj3 = '---'

------

# Exercise 3.2: Objective

As already mentioned in the first notebook, krotov's [`optimize_pulse`](https://krotov.readthedocs.io/en/stable/API/krotov.optimize.html#krotov.optimize.optimize_pulses) method takes so called objectives.
These hold all the information about the goal of the optimization.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Define the objective corresponding to our optimization goal

In [ ]:
objective = '---'

In [ ]:
#q2.hint()

---

# Exercise 3.3: Shaping our guess pulses

In order to demonstrate Krotov’s optimization method, we choose an initial guess consisting of two low intensity and real Blackman pulses which are temporally disjoint. These should look like the following pulses:
<img src="plots/ex2_guess_pulse.png" alt="Here should actually be the Guess Pulses...Ask your advisor if you see this text..." width="700">

&nbsp;&nbsp;&nbsp;
**a)** Write two functions, which will be used as guess pulses for the real and imaginary controls. Try to reproduce the pulses in the plots above.  
&nbsp;&nbsp;&nbsp;
*Hint1: Krotov's
[blackman function](https://krotov.readthedocs.io/en/stable/API/krotov.shapes.html#krotov.shapes.blackman)
in the krotov.shape module might be usefull here.*  
&nbsp;&nbsp;&nbsp;
*Hint2: If you have everything set up, go on to b) and finally plot your results in c).  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
If it does not look as indented, come back to a).*

&nbsp;&nbsp;&nbsp;
*Note: The functions return again a function. This one is the one, that is used for calculating the control!*

In [ ]:
def shape_field_real('--arguments-you-need--'):
    
    #Note, that the function needs to take 2 arguments.
    #You can omitt the 'args' one
    def field_shaped(t, args):
        
        ### how should the field look like
        pass
        ### insert the function here that calculates it
        
    return field_shaped


In [ ]:
def shape_field_imag('--arguments-you-need--'):
    
    #Note, that the function needs to take 2 arguments.
    #You can omitt the 'args' one
    def field_shaped(t, args):
        ### how should the field look like
        pass
        ### insert the function here that calculates it

    return field_shaped

In [ ]:
#q3a.hint()

&nbsp;&nbsp;&nbsp;
**b)** When done, assign the functions to the individual parts of the Hamiltonian

In [ ]:
Ham[1][1] = '...'
Ham[2][1] = '...'
Ham[3][1] = '...'
Ham[4][1] = '...'

&nbsp;&nbsp;&nbsp;
**c)** Verify that everything works as expected by executing the cell below.  
&nbsp;&nbsp;&nbsp;
*Note: You might want to choose more expressive titles in the `plot_pulse` routine*

In [ ]:
def plot_pulse(pulse, tlist, ax, title):
    if callable(pulse):
        pulse = np.array([pulse(t, args=None) for t in tlist])
    ax.plot(tlist, pulse)
    ax.set_xlabel('time')
    ax.set_ylabel('pulse amplitude')
    ax.set_title(title)
     
fig, ax = plt.subplots(2,2)
plot_pulse(Ham[1][1], tlist, ax[0,0], title='Ham[1][1]')
plot_pulse(Ham[2][1], tlist, ax[0,1], title='Ham[2][1]')
plot_pulse(Ham[3][1], tlist, ax[1,0], title='Ham[3][1]')
plot_pulse(Ham[4][1], tlist, ax[1,1], title='Ham[4][1]')
plt.tight_layout()
plt.show(fig)

After having set up everything, let's see how good our guess is!
Therefore we need to simulate the dynamics of the pulse.

&nbsp;&nbsp;&nbsp;
**d)** Use the [`mesolve` function](https://krotov.readthedocs.io/en/stable/API/krotov.objectives.html#krotov.objectives.Objective.mesolve) of your objective to calculate the resulting populations of the individual states.  
&nbsp;&nbsp;&nbsp;
Make sure you give the right expectation operators to the function!

In [ ]:
guess_dynamics = '---'

In [ ]:
#q3d.hint()

Let's see what comes out:

In [ ]:
fig, ax = plt.subplots()
ax.plot(guess_dynamics.times, guess_dynamics.expect[0], label='Projector 1')
ax.plot(guess_dynamics.times, guess_dynamics.expect[1], label='Projector 2')
ax.plot(guess_dynamics.times, guess_dynamics.expect[2], label='Projector 3')
ax.legend()
ax.set_xlabel('time')
ax.set_ylabel('population')
plt.show(fig)

&nbsp;&nbsp;&nbsp;
**e)** Does this make sense?

---

# Exercise 3.4: Specifying the pulse options

Now that our Hamiltonian is completely set up and the objective for our optimization is clear, we need to specify the parameters for the krotov algorithm.
Therefore, we need to set the pulse options for the optimization.

First of all, we define the pulse shape. This needs to be between 0 and 1 and should go to 0 at the beginning and at the end of the time interval, which we take into account. As a first guess, we choose:

In [ ]:
def update_shape(t):
    """Scales the Krotov methods update of the pulse value at the time t"""
    return krotov.shapes.flattop(t,0.,5.,t_rise=.0001,func='sinsq')

&nbsp;&nbsp;&nbsp;
**a)** Play around with the `t_rise` parameter and plot the update shape with the following cell.
Choose a resonable value.  
&nbsp;&nbsp;&nbsp;
You can later also play around with that and see how this changes you optimization.

In [ ]:
fig, ax = plt.subplots()
ax.plot(tlist, np.vectorize(update_shape)(tlist))
ax.set_xlabel('time')
ax.set_ylabel('Update shape')
plt.show(fig)

Now let us continue to and define the pulse options. Unfortunatelly $\lambda_a$ was very cautious estimated and might leed to very slow convergence.  
&nbsp;&nbsp;&nbsp;
**b)** Are you aware of how to change $\lambda_a$? Carefully do that and play around with it after the first optimization in the next exercise

In [ ]:
opt_lambda = 100
pulse_options = {
    Ham[1][1]: dict(lambda_a=opt_lambda, update_shape=update_shape),
    Ham[2][1]: dict(lambda_a=opt_lambda, update_shape=update_shape),
    Ham[3][1]: dict(lambda_a=opt_lambda, update_shape=update_shape),
    Ham[4][1]: dict(lambda_a=opt_lambda, update_shape=update_shape)
}

In [ ]:
#q4b.hint()

---

# Exercise 3.5: The optimization

Finally we can use krotov's optimize_pulses with all the information we build up in the previous examples.

Fill in the following missing values, which are indicated by `'###############'`. Proceed as follows:

&nbsp;&nbsp;&nbsp;
**a)** Recall the structure of the function by using the [docs](https://krotov.readthedocs.io/en/stable/API/krotov.optimize.html#krotov.optimize.optimize_pulses).

&nbsp;&nbsp;&nbsp;
**b)** Which functional (and therefore which `chi_constructor`) do we need here?  
&nbsp;&nbsp;&nbsp;
Check the corresponding section in [Krotov's method](https://krotov.readthedocs.io/en/stable/06_krotovs_method.html#functionals) and choose from the [functionals module](https://krotov.readthedocs.io/en/stable/API/krotov.functionals.html).

&nbsp;&nbsp;&nbsp;
**c)** What do the values for the `check_convergence` and `iter_stop` argument mean?  
&nbsp;&nbsp;&nbsp;
Make a resonable choice here.

&nbsp;&nbsp;&nbsp;
**d)** Maybe your optimization takes quite some time! Adjust the relevant paramters to obtain a better convergence (and thus better results less time).  
&nbsp;&nbsp;&nbsp;
However, take care, that the changes you make are resonable (maybe we want to optimize for an experiment).




In [ ]:
oct_result = krotov.optimize_pulses(
    '#######a#######',
    '#######a#######',
    '#######a#######',
    propagator=krotov.propagators.expm,
    #
    chi_constructor='#######b#######',
    #
    info_hook=krotov.info_hooks.print_table(
        J_T='#######b#######',
        unicode=True,
    ),
    check_convergence=krotov.convergence.Or(
        krotov.convergence.value_below('#######c#######', name='J_T'),
        krotov.convergence.delta_below('#######c#######'),
        krotov.convergence.check_motonic_error,
    ),
    iter_stop='#######c#######',
)


In [ ]:
#q5b.hint()

In [ ]:
#q5c.hint()

In [ ]:
#q5d.hint()

In [ ]:
oct_result

---

# Exercise 3.6: Analysing the results

So now let's see, how our solution looks like

&nbsp;&nbsp;&nbsp;
**a)** Get the resulting objectives from the [oct_result](https://krotov.readthedocs.io/en/stable/API/krotov.result.html) and use mesolve to simulate the dynamics under the optimized pulse (as in 3d)).

In [ ]:
opt_dynamics = '-------'

In [ ]:
#q6a.hint()

After simulating the optimized dynamics we can plot them via

In [ ]:
fig, ax = plt.subplots()
ax.plot(opt_dynamics.times, opt_dynamics.expect[0], label='Projector 1')
ax.plot(opt_dynamics.times, opt_dynamics.expect[1], label='Projector 2')
ax.plot(opt_dynamics.times, opt_dynamics.expect[2], label='Projector 3')
ax.legend()
ax.set_xlabel('time')
ax.set_ylabel('population')
plt.show(fig)

---
  
Now we can also extract the optimized pulses and plot the amplitudes and phases of the pulses.
To do this, you can use the following function, which takes the real and the imaginary part of the pulse and plot the amplitude and the phase:

In [ ]:
def plot_pulse_amplitude_and_phase(pulse_real, pulse_imaginary,tlist):
    ax1 = plt.subplot(211)
    ax2 = plt.subplot(212)
    amplitudes = [np.sqrt(x*x + y*y) for x,y in zip(pulse_real,pulse_imaginary)]
    phases = [np.arctan2(y,x)/np.pi for x,y in zip(pulse_real,pulse_imaginary)]
    ax1.plot(tlist,amplitudes)
    ax1.set_xlabel('time')
    ax1.set_ylabel('pulse amplitude')
    ax2.plot(tlist,phases)
    ax2.set_xlabel('time')
    ax2.set_ylabel('pulse phase (π)')
    plt.show()


---
&nbsp;&nbsp;&nbsp;
**b)** Plot the optimized controls, which are contained in the [oct_result](https://krotov.readthedocs.io/en/stable/API/krotov.result.html).

In [ ]:

print("pump pulse amplitude and phase:")
plot_pulse_amplitude_and_phase(
        "--real-pump-controls--",
        "--imag-pump-controls--", 
        tlist
    )


In [ ]:

print("Stokes pulse amplitude and phase:")
plot_pulse_amplitude_and_phase(
        "--real-stokes-controls--",
        "--imag-stokes-controls--", 
        tlist
    )


---

---

# Bonus exercise

In more realistic physical system, we often have to deal with dissipation. Let's for example consider a spontaneous decay in level $\ket{2}$. This can be a good approximation if the levels $\ket{1}$ and $\ket{3}$ have a decay time, which is much longer than the duration of the pulse.

To prevent the "loss" of population from level $\ket{2}$, we need will need to "tell" krotov how, that  $\ket{2}$ should be avoided. We can do this by adding a phenomenological decay $-i\gamma\ket{2}\bra{2}$ to the Hamiltonian.

Add the dissipation with a loss of 0.5 to the Hamiltonian and do the optimization with again!

*Hint:
Since we have a non-hermitian Hamiltonian, we can no longer use the mesolve routine for the dynamics from QuTiP! Therefore, you need to use 
`obj.propagate` instead. You can use the code below:*

In [ ]:
dynamics = "--Add-objective-here--".propagate(
    tlist, propagator=krotov.propagators.expm, e_ops='--List-of-projectors--'
)
states = "--Add-objective-here--".propagate(
    tlist, propagator=krotov.propagators.expm
)

You can also use the following block to plot the norm and its loss over time: 

In [ ]:
state_norm = lambda i: states.states[i].norm()
states_norm=np.vectorize(state_norm)

fig, ax = plt.subplots()
ax.plot(states.times, states_norm(np.arange(len(states.states))))
ax.set_title('Norm loss', fontsize = 15)
ax.set_xlabel('time')
ax.set_ylabel('state norm')
plt.show(fig)